Creación del script

In [47]:
import pandas as pd
import json

In [2]:
# Cargar el archivo JSON
with open('taylor_swift_spotify.json', 'r') as file:
    data = json.load(file)

# Convertir el JSON a un DataFrame de pandas
df = pd.json_normalize(data)

soloist_data = pd.json_normalize(data=d['programs'], record_path=['works', 'soloists'], 
                                 meta=['id'])
soloist_data.head(10)

# Guardar el DataFrame como un archivo CSV
df.to_csv('archivo.csv', index=False)


Transformación completada. Se ha creado el archivo CSV.


In [48]:
with open('taylor_swift_spotify.json', 'r') as file:
    data = json.load(file)

In [53]:
albums_data = pd.json_normalize(data=data, record_path='albums', 
                                 meta=['artist_id', 'artist_name', 'artist_popularity'])

albums_data.columns

#albums_data.head()

Index(['album_id', 'album_name', 'album_release_date', 'album_total_tracks',
       'tracks', 'artist_id', 'artist_name', 'artist_popularity'],
      dtype='object')

In [52]:
albums_data['track_id'] = albums_data['tracks'].apply(lambda x: x[0]['track_id'])
#albums_data

In [58]:
tracks_data = pd.json_normalize(data=data, record_path=['albums', 'tracks'], 
                                 meta=['artist_id', 'artist_name', 'artist_popularity', ['albums', 'album_id']],
                                 meta_prefix='')

tracks_data.columns

#tracks_data.head()

# 'artist_name', 'artist_popularity', 'album_id', 'album_name', 'album_release_date', 'album_total_tracks'

Index(['disc_number', 'duration_ms', 'explicit', 'track_number',
       'track_popularity', 'track_id', 'track_name',
       'audio_features.danceability', 'audio_features.energy',
       'audio_features.key', 'audio_features.loudness', 'audio_features.mode',
       'audio_features.speechiness', 'audio_features.acousticness',
       'audio_features.instrumentalness', 'audio_features.liveness',
       'audio_features.valence', 'audio_features.tempo', 'audio_features.id',
       'audio_features.time_signature', 'artist_id', 'artist_name',
       'artist_popularity', 'albums.album_id'],
      dtype='object')

In [32]:
tracks = list(tracks_data.columns)
albums = list(albums_data.columns)
en_comun = []

for item in tracks:
    if item in albums:
        en_comun.append(item)

en_comun

['artist_id', 'artist_name', 'artist_popularity']

In [36]:
def flatten_nested_json_df(df):
    df = df.reset_index()
    s = (df.map(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.map(type) == dict).all()
    dict_columns = s[s].index.tolist()
    
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        for col in dict_columns:
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace
        for col in list_columns:
            #print(f"exploding: {col}")
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)
        s = (df[new_columns].map(type) == list).all()
        list_columns = s[s].index.tolist()
        s = (df[new_columns].map(type) == dict).all()
        dict_columns = s[s].index.tolist()
    return df

a_ver = flatten_nested_json_df(albums_data)

a_ver.columns


Index(['index', 'album_id', 'album_name', 'album_release_date',
       'album_total_tracks', 'artist_id', 'artist_name', 'artist_popularity',
       'tracks.disc_number', 'tracks.duration_ms', 'tracks.explicit',
       'tracks.track_number', 'tracks.track_popularity', 'tracks.track_id',
       'tracks.track_name', 'tracks.audio_features.danceability',
       'tracks.audio_features.energy', 'tracks.audio_features.key',
       'tracks.audio_features.loudness', 'tracks.audio_features.mode',
       'tracks.audio_features.speechiness',
       'tracks.audio_features.acousticness',
       'tracks.audio_features.instrumentalness',
       'tracks.audio_features.liveness', 'tracks.audio_features.valence',
       'tracks.audio_features.tempo', 'tracks.audio_features.id',
       'tracks.audio_features.time_signature'],
      dtype='object')

In [33]:
#counties = json_normalize(data, 'counties', ['state', 'shortname'])
#governors = json_normalize(data, 'info', ['state'])

result = tracks_data.merge(albums_data, on='tracks')

KeyError: 'tracks'

In [23]:
data = pd.merge(albums_data, tracks_data, how="left", on=["artist_id"])
data.head(10)

,album_id,album_name,album_release_date,album_total_tracks,tracks,artist_id,artist_name_x,artist_popularity_x,disc_number,duration_ms,...,audio_features.speechiness,audio_features.acousticness,audio_features.instrumentalness,audio_features.liveness,audio_features.valence,audio_features.tempo,audio_features.id,audio_features.time_signature,artist_name_y,artist_popularity_y
0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,212600,...,0.0327,0.00942,0.000037,0.3670,0.685,116.998,4WUepByoeqcedHoYhSNHRt,4.0,Taylor Swift,120
1,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,231833,...,0.0670,5,0,0.1680,0.701,96.057,0108kcWLnn2HlH2kedi1gn,4.0,Taylor Swift,120
2,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,231000,...,0.0397,0.000421,0.0197,0.0899,0.305,94.868,3Vpk1hfMAQme8VJ0SNRSkd,4.0,Taylor Swift,120
3,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,235800,...,0.0447,-0.000537,0.000056,0.3850,0.206,92.021,1OcSfkeCg9hRC2sFKB4IMJ,4.0,Taylor Swift,120
4,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,193289,...,0.0317,0.000656,0,0.1310,0.520,96.997,2k0ZEeAqzvYMcx9Qt5aClQ,4.0,Taylor Swift,120
5,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,219209,...,0.0729,0.0121,0.000022,0.3590,0.917,160.058,50yNTF0Od55qnHLxYsA5Pw,4.0,Taylor Swift,120
6,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,207650,...,0.0439,-0.00354,0.000013,0.0687,0.539,118.009,3FxJDucHWdw6caWTKO5b23,4.0,Taylor Swift,120
7,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,211103,...,0.1940,0.0362,0,0.3050,0.363,169.971,7oZONwFiFIErZcXAtTu7FY,4.0,Taylor Swift,120
8,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,220433,...,0.0656,0.0436,0.000072,0.1120,0.514,139.985,27exgla7YBw9DUNNcTIpjy,4.0,Taylor Swift,120
9,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2023-10-27,22,"[{'disc_number': 1, 'duration_ms': 212600, 'ex...",06HL4z0CvFAxyc27GX,Taylor Swift,120,1,247533,...,0.0515,0.00196,0.000011,0.0939,0.538,119.997,733OhaXQIHY7BKtY3vnSkn,4.0,Taylor Swift,120


In [19]:
# Converting JSON data to a pandas DataFrame
df = pd.read_json("taylor_swift_spotify.json")

df

,artist_id,artist_name,artist_popularity,albums
0,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '1o59UpKw81iHR0HPiSkJR0', 'album_..."
1,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '64LU4c1nfjz1t4VnGhagcg', 'album_..."
2,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '5AEDGbliTTfjOB8TSm1sxt', 'album_..."
3,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '1fnJ7k0bllNfL1kVdNVW1A', 'album_..."
4,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '3lS1y25WAhcqJDATJK70Mq', 'album_..."
5,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '151w1FgRZfnKZA9FEcg9Z3', 'album_..."
6,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '6kZ42qRrzov54LcAk4onW9', 'album_..."
7,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '4hDok0OAJd57SGIT8xuWJH', 'album_..."
8,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '6AORtDjduMM3bupSWzbTSG', 'album_..."
9,06HL4z0CvFAxyc27GX,Taylor Swift,120,"{'album_id': '2Xoteh7uEpea4TohMxjtaq', 'album_..."
